# Compare metrics between all 'Triplets' and 'Permutations'



In [1]:
import itertools as itt
import pathlib as pl
from configparser import ConfigParser
from textwrap import fill

import joblib as jl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scipy.stats as sst
import seaborn as sns
from cycler import cycler
from ipywidgets import interact, fixed

import src.visualization.fancy_plots as fplt
from src.data.cache import set_name
from src.metrics.reliability import signal_reliability
from src.data.rasters import raster_from_sig
import src.data.dPCA as cdPCA
from src.data.load import load
from src.metrics.dprime import flip_dprimes

[nems.configs.defaults INFO] Saving log messages to /tmp/nems/NEMS 2020-12-23 143844.log


In [2]:
plt.style.use('dark_background')
# modify figure color cycler back to the default one
color_cycler = cycler(color=['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd',
                            '#8c564b', '#e377c2', '#7f7f7f', '#bcbd22', '#17becf'])

params = {'legend.fontsize': 'medium',
          'axes.labelsize': 15,
          'axes.titlesize': 20,
          'axes.spines.top': False,
          'axes.spines.right': False,
          'axes.prop_cycle': color_cycler,
          'xtick.labelsize': 11,
          'ytick.labelsize': 11,
          'lines.markersize': 8,
          'figure.titlesize': 30,
          'figure.figsize': [6,6],
          'figure.autolayout':True,
          'svg.fonttype': 'none',
          'font.sans-serif': 'Arial'
          }

plt.rcParams.update(params)


In [3]:
config = ConfigParser()
config.read_file(open(pl.Path().cwd().parent / 'config' / 'settings.ini'))

In [6]:
meta = {'reliability': 0.1,  # r value
        'smoothing_window': 0,  # ms
        'raster_fs': 30,
        'transitions': ['silence', 'continuous', 'similar', 'sharp'],
        'montecarlo': 1000,
        'zscore': True,
        'dprime_absolute': None}

# triplets DF
summary_DF_file = pl.Path(config['paths']['analysis_cache']) / 'DF_summary' / set_name(meta)
trpDF = jl.load(summary_DF_file)
# create the id_probe pair for
trpDF['id_probe'] = trpDF['cellid'].fillna(value=trpDF['siteid'])
trpDF['id_probe'] = trpDF[['id_probe', 'probe']].agg('_'.join, axis=1)
trpDF['stim'] = 'triplets'


In [7]:
meta = {'reliability': 0.1,  # r value
        'smoothing_window': 0,  # ms
        'raster_fs': 30,
        'transitions': [0, 1, 2, 3, 4],
        'montecarlo': 1000,
        'zscore': True,
        'dprime_absolute': None}

# permutations DF
summary_DF_file = pl.Path(config['paths']['analysis_cache']) / 'prm_summary_DF_v2' / set_name(meta)
permDF = jl.load(summary_DF_file)
#create the id_probe pair for
permDF['id_probe'] = permDF['cellid'].fillna(value=permDF['siteid'])
permDF['id_probe'] = permDF[['id_probe', 'probe']].agg('_'.join, axis=1)
permDF['stim'] = 'permutations'


In [10]:
DF = pd.concat([trpDF, permDF])
print(DF.columns)
print(DF.shape)


In [12]:
# common parameters
analysis = ['SC', 'dPCA', 'pdPCA','fdPCA']
analysis = ['SC']

ff_parameter = DF.parameter.isin(['significant_abs_sum', 'significant_abs_mass_center'])
ff_analysis = DF.analysis.isin(analysis)
ff_source = DF.source == 'dprime'
ff_probe = DF.probe == 'mean'
ff_transpair= DF.transition_pair == 'mean'
ff_good = DF.value > 0 # empirical good value to filter out garbage cells

filtered = DF.loc[ff_parameter & ff_analysis & ff_source & ff_probe & ff_transpair & ff_good, :]

       analysis        cellid  goodness                    parameter probe  \
452          SC  CRD003b-02-2       NaN          significant_abs_sum  mean   
907          SC  CRD003b-03-1       NaN          significant_abs_sum  mean   
909          SC  CRD003b-03-1       NaN  significant_abs_mass_center  mean   
1362         SC  CRD003b-05-1       NaN          significant_abs_sum  mean   
1364         SC  CRD003b-05-1       NaN  significant_abs_mass_center  mean   
1817         SC  CRD003b-06-1       NaN          significant_abs_sum  mean   
1819         SC  CRD003b-06-1       NaN  significant_abs_mass_center  mean   
2272         SC  CRD003b-07-1       NaN          significant_abs_sum  mean   
2274         SC  CRD003b-07-1       NaN  significant_abs_mass_center  mean   
3182         SC  CRD003b-14-1       NaN          significant_abs_sum  mean   
3184         SC  CRD003b-14-1       NaN  significant_abs_mass_center  mean   
4092         SC  CRD003b-21-1       NaN          significant_abs

In [ ]:
pivoted = filtered.pivot_table(index=['siteid', 'parameter'],
                               columns='analysis', values='value').dropna().reset_index()

g = sns.lmplot(x='pdPCA', y='fdPCA', col='parameter', data=pivoted,
                 sharex=False, sharey=False,
                 scatter_kws={'color':'white'}, line_kws={'color':'white'})
g.set_titles('{col_name}')
g.fig.tight_layout()

for ax in g.axes.flat:
    _ = fplt.unit_line(ax)

